In [1]:
#import dependencies 
from numpy.random import seed
from numpy.random import randn
from scipy import stats
import numpy as np

import math  

import seaborn as sns
sns.set(style="darkgrid")
from matplotlib import pyplot as plt

import pandas as pd
import os

In [400]:
#check list of files in data 
print(os.listdir("./data/open_cloud"))

['.DS_Store', '50x_m5d_iperf_11-16-2019.csv', '50x_m5d_y-cruncher_open_cloud_11-16-2019.csv', '30x_m5d_11-14-2019', '50x_m5d_sysbench_open_cloud_11-16-2019-us-east-1b-2.csv', '50x_m5d_pgbench.csv']


In [455]:
#DON'T FORGET TO SET THIS!

#change experiment type and csv data...
experiment_type = 'pgbench'
data_name = '50x_m5d_pgbench.csv'

In [456]:
df_data = pd.read_csv('./data/open_cloud/' + data_name);

In [457]:
df_data.describe()

,wallTime,clients,threads,scaleFactor,transactions
count,500.000000,500.0,500.0,500.0,500.000000
mean,60.228266,10.0,10.0,1.0,152748.188000
std,0.073263,0.0,0.0,0.0,9269.536107
min,60.126505,10.0,10.0,1.0,135175.000000
25%,60.172952,10.0,10.0,1.0,144696.750000
50%,60.209942,10.0,10.0,1.0,149481.000000
75%,60.266476,10.0,10.0,1.0,160833.250000
max,60.621241,10.0,10.0,1.0,168688.000000


In [458]:
#iperf remove trailing text
if (experiment_type == 'iperf'):
    df_data['bandwidthUpload'] = df_data['bandwidthUpload'].str.split(" ").str[0].astype('float64')
    df_data['bandwitdhDownload'] = df_data['bandwitdhDownload'].str.split(" ").str[0].astype('float64')
    #df_data.dropna(inplace = True)

In [459]:
df_data.tail()

,instanceID,experimentID,instanceType,wallTime,clients,threads,scaleFactor,transactionsType,queryMode,duration,transactions,mountPoint
495,i-0f83d46715e5b7ba9,1573943301-0,m5d.large,60.245624,10,10,1,TPC-B (sort of),simple,60 s,166585,data_directory = '/var/lib/postgresql/9.5/main'
496,i-0f83d46715e5b7ba9,1573943301-0,m5d.large,60.292593,10,10,1,TPC-B (sort of),simple,60 s,166629,data_directory = '/var/lib/postgresql/9.5/main'
497,i-0f83d46715e5b7ba9,1573943301-0,m5d.large,60.263600,10,10,1,TPC-B (sort of),simple,60 s,166794,data_directory = '/var/lib/postgresql/9.5/main'
498,i-0f83d46715e5b7ba9,1573943301-0,m5d.large,60.309227,10,10,1,TPC-B (sort of),simple,60 s,166379,data_directory = '/var/lib/postgresql/9.5/main'
499,i-0f83d46715e5b7ba9,1573943301-0,m5d.large,60.290344,10,10,1,TPC-B (sort of),simple,60 s,166910,data_directory = '/var/lib/postgresql/9.5/main'


In [460]:
if (experiment_type == 'iperf'):
    #adjust gigbite values 'up' assuming all mbs values are over 100mb
    df_data['bandwitdhDownload'] = np.where(df_data['bandwitdhDownload'] > 100, df_data['bandwitdhDownload'], df_data['bandwitdhDownload']*1000)
    df_data['bandwidthUpload'] = np.where(df_data['bandwidthUpload'] > 100, df_data['bandwidthUpload'], df_data['bandwidthUpload']*1000)
    
    #add total bandwidth column
    df_data['total'] = df_data['bandwidthUpload'] + df_data['bandwitdhDownload']
    
    #sort by setId, vmId
    df_data = df_data.sort_values(by =['setId', 'vmId'], ascending = False)
    
    #reset index
    df_data.reset_index(drop=True, inplace= True)

In [461]:
#set nth column #change depending on data set...
if (experiment_type == 'iperf'):
    nth_column = 11
    df_data = df_data[:551] #drop every run except first run
elif (experiment_type != 'pgbench'):
    nth_column = 10

In [462]:
#drop every nth column
#drop first row of each vm run since they are outliers (except pgbench)
if (experiment_type != 'pgbench'):
    df_data = df_data[df_data.index % nth_column != 0]

In [467]:
#check if correct columns are dropped..
df_data.head(14)

,instanceID,experimentID,instanceType,wallTime,clients,threads,scaleFactor,transactionsType,queryMode,duration,transactions,mountPoint
0,i-04b9c2f75a375d7e5,1573943301-0,m5d.large,60.302226,10,10,1,TPC-B (sort of),simple,60 s,167389,data_directory = '/var/lib/postgresql/9.5/main'
1,i-04b9c2f75a375d7e5,1573943301-0,m5d.large,60.276026,10,10,1,TPC-B (sort of),simple,60 s,167447,data_directory = '/var/lib/postgresql/9.5/main'
2,i-04b9c2f75a375d7e5,1573943301-0,m5d.large,60.208249,10,10,1,TPC-B (sort of),simple,60 s,167434,data_directory = '/var/lib/postgresql/9.5/main'
3,i-04b9c2f75a375d7e5,1573943301-0,m5d.large,60.294649,10,10,1,TPC-B (sort of),simple,60 s,168173,data_directory = '/var/lib/postgresql/9.5/main'
4,i-04b9c2f75a375d7e5,1573943301-0,m5d.large,60.227326,10,10,1,TPC-B (sort of),simple,60 s,168140,data_directory = '/var/lib/postgresql/9.5/main'
5,i-04b9c2f75a375d7e5,1573943301-0,m5d.large,60.170879,10,10,1,TPC-B (sort of),simple,60 s,167926,data_directory = '/var/lib/postgresql/9.5/main'
6,i-04b9c2f75a375d7e5,1573943301-0,m5d.large,60.170726,10,10,1,TPC-B (sort of),simple,60 s,167871,data_directory = '/var/lib/postgresql/9.5/main'
7,i-04b9c2f75a375d7e5,1573943301-0,m5d.large,60.172466,10,10,1,TPC-B (sort of),simple,60 s,168380,data_directory = '/var/lib/postgresql/9.5/main'
8,i-04b9c2f75a375d7e5,1573943301-0,m5d.large,60.172143,10,10,1,TPC-B (sort of),simple,60 s,168249,data_directory = '/var/lib/postgresql/9.5/main'
9,i-04b9c2f75a375d7e5,1573943301-0,m5d.large,60.179965,10,10,1,TPC-B (sort of),simple,60 s,168688,data_directory = '/var/lib/postgresql/9.5/main'


In [475]:
#group by instanceID and take the mean of each column
df = df_data.groupby('instanceID').mean()

In [476]:
#groupby instanceID and take the standard deviation each column
df_std = df_data.groupby('instanceID').std()

In [477]:
df.head()

,wallTime,clients,threads,scaleFactor,transactions
instanceID,,,,,
i-0062d028d4b1b0c17,60.259524,10.0,10.0,1.0,159093.7
i-0069a2fb2881e526e,60.268572,10.0,10.0,1.0,145564.0
i-0096e0731defa5bcf,60.257457,10.0,10.0,1.0,144451.6
i-0098bacd16b57b4dc,60.268732,10.0,10.0,1.0,165235.5
i-00a374920f339188c,60.177961,10.0,10.0,1.0,165764.5


In [478]:
#add stds to main columns
if (experiment_type == 'iperf'):
    df['stdUp'] = df_std['bandwidthUpload']
    df['stdDown'] = df_std['bandwitdhDownload']
    df['stdTotal'] = df_std['total']
elif (experiment_type == 'y-cruncher'):
    df['std_computationTime'] = df_std['computationTime']
    df['std_benchmarkTime'] = df_std['benchmarkTime']
    df['std_wallTime'] = df_std['wallTime']
elif (experiment_type == 'sysbench'):
    df['std_wallTime'] = df_std['wallTime']
elif (experiment_type == 'pgbench'):
    df['std_wallTime'] = df_std['wallTime']
    df['std_transactions'] = df_std['transactions']
else:
    print('illegal benchmark name')

In [479]:
df.tail()

,wallTime,clients,threads,scaleFactor,transactions,std_wallTime,std_transactions
instanceID,,,,,,,
i-0ece06eab633a3526,60.269408,10.0,10.0,1.0,143687.2,0.039654,246.898450
i-0f0e24c09425eac71,60.334347,10.0,10.0,1.0,151546.2,0.147230,3673.404863
i-0f83d46715e5b7ba9,60.298434,10.0,10.0,1.0,166632.6,0.052757,187.065170
i-0fe52617ed49f921d,60.237538,10.0,10.0,1.0,142223.0,0.095654,319.146431
i-0ff32a7bce1e3a734,60.168594,10.0,10.0,1.0,148156.9,0.021631,1723.681103


In [480]:
if (experiment_type == 'sysbench'):
    df.drop(columns =['thread-num'], inplace = True)
elif (experiment_type == 'iperf'):
    df.drop(columns =['setId', 'experimentID'], inplace = True)
elif (experiment_type == 'pgbench'):
    df.drop(columns =['clients', 'threads', 'scaleFactor'], inplace=True)

In [481]:
df.head()

,wallTime,transactions,std_wallTime,std_transactions
instanceID,,,,
i-0062d028d4b1b0c17,60.259524,159093.7,0.100973,387.417676
i-0069a2fb2881e526e,60.268572,145564.0,0.031911,243.825074
i-0096e0731defa5bcf,60.257457,144451.6,0.062333,338.681824
i-0098bacd16b57b4dc,60.268732,165235.5,0.045580,169.238326
i-00a374920f339188c,60.177961,165764.5,0.005262,322.349103


In [482]:
#print to csv
df.to_csv(path_or_buf='./data/summary/' + data_name + '-Summary.csv')

In [483]:
df.describe()

,wallTime,transactions,std_wallTime,std_transactions
count,50.000000,50.000000,50.000000,50.000000
mean,60.228266,152748.188000,0.047355,790.629115
std,0.049633,9266.117496,0.032372,1089.087960
min,60.146381,135802.800000,0.002284,155.456639
25%,60.189266,144613.150000,0.023627,248.760988
50%,60.217629,150072.300000,0.045540,337.063931
75%,60.268692,159909.150000,0.062269,523.353530
max,60.337281,167969.700000,0.147230,3964.703858


In [52]:
#graph the probablity of getting "better vm"